In [ ]:
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif')
plt.rc('font', size=14)
plt.rc('axes', titlesize=14)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('lines', markersize=10)

In [ ]:
de_train_dd = dd.read_parquet('datasets/de_train.parquet')
adata_train_dd = dd.read_parquet('datasets/adata_train.parquet')
multiome_train_dd = dd.read_parquet('datasets/multiome_train.parquet')

adata_obs_meta = pd.read_csv('datasets/adata_obs_meta.csv')
multiome_obs_meta = pd.read_csv('datasets/multiome_obs_meta.csv')
multiome_var_meta = pd.read_csv('datasets/multiome_var_meta.csv')
id_map = pd.read_csv('datasets/id_map.csv')

de_train = de_train_dd.compute()
adata_train = adata_train_dd.compute()
multiome_train = multiome_train_dd.compute()

In [ ]:
# rename in de_train
for i in range(len(de_train['sm_name'])):
    if de_train['sm_name'][i] != '5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine' and de_train['sm_name'][i] != '2-(4-Amino-5-iodo-7-pyrrolo[2,3-d]pyrimidinyl)-5-(hydroxymethyl)oxolane-3,4-diol':
        de_train['sm_name'][i] = de_train['sm_name'][i].split('(')[0].strip()
for i in range(len(de_train['sm_name'])):
    if de_train['sm_name'][i] != '5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine' and de_train['sm_name'][i] != '2-(4-Amino-5-iodo-7-pyrrolo[2,3-d]pyrimidinyl)-5-(hydroxymethyl)oxolane-3,4-diol':
        de_train['sm_name'][i] = de_train['sm_name'][i].split(';')[0].strip()

In [ ]:
adata_train.head()

In [ ]:
de_train.head()

In [ ]:
features_columns = ["cell_type", "sm_name", "sm_lincs_id", "SMILES", "control"]
targets = de_train.drop(columns=features_columns)
features = pd.DataFrame(de_train, columns=features_columns)

In [ ]:
features.head()

In [ ]:
targets.head()

In [ ]:
# Plot distribution of cell types in the training data
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
# pie chart and bar chart
ax[0].pie(features["cell_type"].value_counts(), labels=features["cell_type"].value_counts().index, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 14}, colors=["grey", "lightgrey", "darkgrey"])
ax[0].set_title("Distribution of cell types in the training data")
ax[1].bar(features["cell_type"].value_counts().index, features["cell_type"].value_counts(), edgecolor="black", facecolor="white", hatch="///")
ax[1].set_xlabel("Cell type")
ax[1].set_ylabel("Number of cells")
ax[1].set_title("Distribution of cell types in the training data")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Print column names of all datasets
print("adata_train columns:", adata_train.columns)
print("multiome_train columns:", multiome_train.columns)
print("adata_obs_meta columns:", adata_obs_meta.columns)
print("multiome_obs_meta columns:", multiome_obs_meta.columns)
print("multiome_var_meta columns:", multiome_var_meta.columns)
print("id_map columns:", id_map.columns)

In [ ]:
multiome_train.head()

In [ ]:
# Aggregating adata_train and multiome_train
agg_func = {'count': ['mean', 'sum'], 'normalized_count': ['mean', 'sum']}
adata_train_grouped = adata_train.groupby(['obs_id']).agg(agg_func).reset_index()
multiome_train_grouped = multiome_train.groupby(['obs_id']).agg(agg_func).reset_index()

In [ ]:
adata_train_grouped.head()

In [ ]:
multiome_train_grouped.head()

In [ ]:
# Flatten the multi-level column names in adata_train_grouped and multiome_train_grouped
adata_train_grouped.columns = ['_'.join(col).strip() for col in adata_train_grouped.columns.values]
multiome_train_grouped.columns = ['_'.join(col).strip() for col in multiome_train_grouped.columns.values]

# Rename the 'obs_id_' column back to 'obs_id' to make it ready for the merge
adata_train_grouped.rename(columns={'obs_id_': 'obs_id'}, inplace=True)
multiome_train_grouped.rename(columns={'obs_id_': 'obs_id'}, inplace=True)

# Join aggregated dataframes with metadata
adata_full = pd.merge(adata_train_grouped, adata_obs_meta, on='obs_id', how='left')
multiome_full = pd.merge(multiome_train_grouped, multiome_obs_meta, on='obs_id', how='left')

In [ ]:
adata_full.head()

In [ ]:
multiome_full.head()

In [ ]:
# Aggregating by 'cell_type' and 'sm_name' or 'donor_id'
agg_func_meta = {'count_mean': 'mean', 'count_sum': 'sum', 'normalized_count_mean': 'mean', 'normalized_count_sum': 'sum'}
adata_summary = adata_full.groupby(['cell_type', 'sm_name']).agg(agg_func_meta).reset_index()
multiome_summary = multiome_full.groupby(['cell_type', 'donor_id']).agg(agg_func_meta).reset_index()

In [ ]:
adata_summary.head()

In [ ]:
multiome_summary.head()

In [ ]:
feature_enriched = pd.DataFrame()

In [ ]:
features_enriched = pd.merge(features, adata_summary, on=['cell_type', 'sm_name'], how='left')
features_enriched = pd.merge(features_enriched, multiome_summary, on=['cell_type'], how='left')  # Donor ID can be included if it aligns

In [ ]:
features_enriched['sm_name'] = features_enriched['sm_name'].replace('O-Demethylated Adapalene', 'O-Desmethyl Adapalene')
# Rename sm_name IN1451 to 2-(4-Amino-5-iodo-7-pyrrolo[2,3-d]pyrimidinyl)-5-(hydroxymethyl)oxolane-3,4-diol
features_enriched['sm_name'] = features_enriched['sm_name'].replace('IN1451',
                                                                    '2-(4-Amino-5-iodo-7-pyrrolo[2,3-d]pyrimidinyl)-5-(hydroxymethyl)oxolane-3,4-diol')

In [ ]:
features_enriched.head()

**Mean Counts** (```count_mean_x```, ```count_mean_y```): In transcriptomics, the mean count of gene expression can be a significant factor. Genes that are generally expressed more can be more easily regulated and have higher chances of being co-expressed with other genes. This concept is supported by statistical methods used in transcriptome analysis, such as edgeR or DESeq2, where mean expression levels serve as an essential element in the models (Robinson et al., 2010, Bioinformatics; Love et al., 2014, Genome Biology).

**Sum Counts** (```count_sum_x```, ```count_sum_y```): The sum of counts could signify the overall activity level of the genome in different cells or under different conditions. A higher sum could mean that the cell is in a more "active" state. While this is a more aggregate measure, it can be useful when coupled with more specific measures.

**Normalized Mean Counts** (```normalized_count_mean_x```, ```normalized_count_mean_y```): Gene expression is often normalized to make it comparable across samples. This is especially important when different samples have different sequencing depths. Normalization can be done in various ways, like Transcripts Per Million (TPM) or Fragments Per Kilobase Million (FPKM). These measures make data across samples directly comparable (Wagner et al., 2012, Theory in Biosciences).

**Normalized Sum Counts** (```normalized_count_sum_x```, ```normalized_count_sum_y```): Just like the normalized mean, the sum of all normalized counts could provide an alternative measure of overall genetic activity, making it more comparable across different cells or conditions.

**Comparing across datasets** (```adata_train``` vs ```multiome_train```): Different sequencing technologies or experimental conditions can result in subtle differences in gene expression profiles. Therefore, using features from both datasets might allow the model to capture nuances that could be missing if you relied on only one dataset (Stegle et al., 2015, Nature Reviews Genetics).

In [ ]:
# Print column names of features_enriched
print("features_enriched columns:", features_enriched.columns)

In [ ]:
final_prediction = pd.DataFrame(id_map, columns=["cell_type", "sm_name"])

In [ ]:
final_prediction.head()

In [ ]:
# features_enriched to csv
features_enriched.to_csv('datasets/features_enriched.csv', index=False)

In [ ]:
final_prediction.to_csv('datasets/final_prediction.csv', index=False)

In [ ]:
targets